In [1]:
from datasets import load_dataset
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from json_repair import repair_json
import pandas as pd
from prutils.prevaluation import PrEvaluation

dataset = load_dataset("jingjietan/essays-big5", cache_dir="../datasets")

name = "essays_gpt-3.5-turbo-0125"

In [11]:
def get_single_sample_json(id,user_text):
    messages = [
        {
            "role": "system",
            "content": (
                "Analyze the user's personality using the Big 5 framework based on the given text.\n"
                "Output the result in this JSON format:\n"
                "{\n"
                "  'dimensions': {\n"
                "    'Openness': {'evidence': '', 'label': '0/1'},\n"
                "    'Conscientiousness': {'evidence': '', 'label': '0/1'},\n"
                "    'Extraversion': {'evidence': '', 'label': '0/1'},\n"
                "    'Agreeableness': {'evidence': '', 'label': '0/1'},\n"
                "    'Neuroticism': {'evidence': '', 'label': '0/1'}\n"
                "  }\n"
                "}\n\n"
                "For each dimension, provide:\n"
                "- evidence: Analysis of the text reflecting the trait. Give examples (content and/or tone) from the user's message. Avoid using punctuation\n"
                "- label: Provide only a value (0 for low; or 1 for high) to indicate the traits.\n\n"
            )
        },
        {
            "role": "user",
            "content": user_text
        }
    ]



    data = {
        "custom_id": str(id),
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-3.5-turbo-0125",
            "messages": messages,
            "response_format": { "type": "json_object" },
            "max_tokens": 500,
        }
    }
    
    return data

In [ ]:
testset = dataset['test']
testset = pd.DataFrame(testset)

# create .jsonl file
with open(name + '.jsonl', 'w') as f:
    for _, row in testset.iterrows():
        data = get_single_sample_json(row['__index_level_0__'], row['text'])
        f.write(json.dumps(data))
        f.write('\n')
print(len(testset))

In [ ]:
# submit batch to open ai using jsonl file
from openai import OpenAI
client = OpenAI(
    api_key="___"
)

input_file = client.files.create(
  file=open(name+".jsonl", "rb"),
  purpose="batch"
)

print(input_file.id)

In [8]:
batch = client.batches.create(
    input_file_id=input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

In [ ]:
batch = client.batches.retrieve(batch.id)
print(batch)

In [7]:
file_response = client.files.content(batch.output_file_id)
#print(file_response.text)

In [12]:
df = pd.DataFrame(columns=["id","o_evi", "o_type", "c_evi", "c_type", "e_evi", "e_type", "a_evi", "a_type", "n_evi", "n_type", "O", "C", "E", "A", "N"])

for line in file_response.text.strip().split('\n'):
    line = json.loads(line)
    custom_id = line["custom_id"]
    content = line["response"]["body"]["choices"][0]["message"]["content"]

    O = testset[testset["__index_level_0__"] == int(custom_id)]["O"].values[0]
    C = testset[testset["__index_level_0__"] == int(custom_id)]["C"].values[0]
    E = testset[testset["__index_level_0__"] == int(custom_id)]["E"].values[0]
    A = testset[testset["__index_level_0__"] == int(custom_id)]["A"].values[0]
    N = testset[testset["__index_level_0__"] == int(custom_id)]["N"].values[0]

    try:
        content = repair_json(content)
        content = json.loads(content)
        content = content["dimensions"]

        o_evi = content["Openness"]["evidence"]
        o_type = content["Openness"]["label"]
        c_evi = content["Conscientiousness"]["evidence"]
        c_type = content["Conscientiousness"]["label"]
        e_evi = content["Extraversion"]["evidence"]
        e_type = content["Extraversion"]["label"]
        a_evi = content["Agreeableness"]["evidence"]
        a_type = content["Agreeableness"]["label"]
        n_evi = content["Neuroticism"]["evidence"]
        n_type = content["Neuroticism"]["label"]

        #make to pandas
        df = df.append({"id":custom_id, "o_evi": o_evi, "o_type": o_type, "c_evi": c_evi, "c_type": c_type, "e_evi": e_evi, "e_type": e_type, "a_evi": a_evi, "a_type": a_type, "n_evi": n_evi, "n_type": n_type, "O":O, "C":C, "E":E, "A":A, "N":N}, ignore_index=True)
    except:
        #df = df.append({"id":custom_id,"e_i_evi": "", "e_i_type": "", "s_n_evi": "", "s_n_type": "", "t_f_evi": "", "t_f_type": "", "j_p_evi": "", "j_p_type": "", "O":O, "C":C, "E":E, "A":A}, ignore_index=True)
        # append the content to a txt file
        with open(name + '_error.txt', 'a') as f:
            f.write(str(content))
            f.write('\n\n')

In [13]:
df["o_type"] = df["o_type"].apply(lambda x: 1 if str(x) == "1" else 0 if str(x).upper() == "0" else -1).astype(int)
df["c_type"] = df["c_type"].apply(lambda x: 1 if str(x) == "1" else 0 if str(x).upper() == "0" else -1).astype(int)
df["e_type"] = df["e_type"].apply(lambda x: 1 if str(x) == "1" else 0 if str(x).upper() == "0" else -1).astype(int) 
df["a_type"] = df["a_type"].apply(lambda x: 1 if str(x) == "1" else 0 if str(x).upper() == "0" else -1).astype(int)
df["n_type"] = df["n_type"].apply(lambda x: 1 if str(x) == "1" else 0 if str(x).upper() == "0" else -1).astype(int)

#make int
df["o_type"] = df["o_type"].astype(int)
df["c_type"] = df["c_type"].astype(int)
df["e_type"] = df["e_type"].astype(int)
df["a_type"] = df["a_type"].astype(int)
df["n_type"] = df["n_type"].astype(int)

df["O"] = df["O"].astype(int)
df["C"] = df["C"].astype(int)
df["E"] = df["E"].astype(int)
df["A"] = df["A"].astype(int)
df["N"] = df["N"].astype(int)



In [14]:
# save to csv
df.to_csv(name + '.csv', index=False)

In [15]:
# #read from csv
# import pandas as pd
# df = pd.read_csv(name + '.csv')


In [ ]:
O = PrEvaluation("O")
O.push([df["o_type"].to_list()],[df["O"].to_list()])
O.print_performance()

C = PrEvaluation("C")
C.push([df["c_type"].to_list()],[df["C"].to_list()])
C.print_performance()

E = PrEvaluation("E")
E.push([df["e_type"].to_list()],[df["E"].to_list()])
E.print_performance()

A = PrEvaluation("A")
A.push([df["a_type"].to_list()],[df["A"].to_list()])
A.print_performance()

N = PrEvaluation("N")
N.push([df["n_type"].to_list()],[df["N"].to_list()])
N.print_performance()

In [ ]:
#chekc unique df["N"].to_list()
print(df["n_type"].unique())